# Import

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import requests
import json
from tqdm.notebook import tqdm

# Read in data and API key

In [2]:
k = open('../data/api_keys.json')
keys = json.load(k)
api_key = keys['google_maps']

In [3]:
referrals_big = pd.read_csv('../data/neo4j/referrals_big.csv')

In [4]:
referrals_big['address_from'].nunique()

23600

In [5]:
referrals_big['address_to'].nunique()

23600

# Geocode using Google Geocode API

## Geocode "from" addresses

In [6]:
from_addresses = referrals_big['address_from'].unique()

In [ ]:
#geocode_dict_from = {}

In [40]:
for address in tqdm(from_addresses[22896:]):
    endpoint = 'https://maps.googleapis.com/maps/api/geocode/json'
    params = {'address': address,
             'key': api_key}

    response = requests.get(endpoint, params)
    geocode_dict_from[address] = response.json()['results'][0]['geometry']['location']

  0%|          | 0/704 [00:00<?, ?it/s]

In [39]:
from_addresses[22894:22898]

array(['61 MEMORIAL MEDICAL PKWY, SUITE 2812, PALM COAST, FL 32164',
       '1454 LAKE SIDE AVE, DAVENPORT, FL 33837',
       '601 E ROLLINS ST, DEPT OF PATHOLOGY, ORLANDO, FL 32803',
       '1305 LAKELAND HILLS BLVD, LAKELAND, FL 33805'], dtype=object)

In [45]:
len(geocode_dict_from.keys())

23600

In [46]:
from_address_geocodes = pd.DataFrame(geocode_dict_from).transpose().reset_index().rename(columns = {'index':'address_from'})
from_address_geocodes

,address_from,lat,lng
0,"302 N CONGRESS BLVD, SMITHVILLE, TN 37166",35.969171,-85.807472
1,"3024 BUSINESS PARK CIR, GOODLETTSVILLE, TN 37072",36.332108,-86.700830
2,"1801 1ST AVE S, BIRMINGHAM, AL 35233",33.510529,-86.805950
3,"1301 22ND AVE S, NASHVILLE, TN 37232",36.141268,-86.801869
4,"5811 CROSSINGS BLVD, ANTIOCH, TN 37013",36.037068,-86.641422
...,...,...,...
23595,"405 W 16TH ST, HOPE, AR 71801",33.653368,-93.591908
23596,"16502 LAWSON RD STE A, LITTLE ROCK, AR 72210",34.704582,-92.455895
23597,"1113 N CASTLE HEIGHTS AVE, SUITE D, LEBANON, T...",36.225734,-86.293254
23598,"275 JACKSON MEADOWS DR STE 101, HERMITAGE, TN ...",36.219183,-86.601242


## Geocode "to" addresses

In [ ]:
to_addresses = referrals_big['address_to'].unique()

In [ ]:
geocode_dict_to = {}

for address in tqdm(to_addresses):
    endpoint = 'https://maps.googleapis.com/maps/api/geocode/json'
    params = {'address': address,
             'key': api_key}

    response = requests.get(endpoint, params)
    geocode_dict_to[address] = response.json()['results'][0]['geometry']['location']

In [ ]:
to_address_geocodes = pd.DataFrame(geocode_dict_to).transpose().reset_index().rename(columns = {'index':'address_to'})
to_address_geocodes

# Write to CSV
So we don't accidentally use up this Google API credit

In [47]:
to_address_geocodes.to_csv('../data/geocodes/to_address_geocodes_big.csv', index = False)
from_address_geocodes.to_csv('../data/geocodes/from_address_geocodes_big.csv', index = False)